La mayor parte de este ETL fue hecha en databricks adjunto la copia del cuaderno que estara activa por 6 meses segun informo....... databrics

https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3189699808379647/323412156151872/2927535050053208/latest.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql.functions import isnan, when, count, concat, lit, col, explode,replace, regexp_replace, udf,length, lower, regexp_replace, array_union, array_join
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, BooleanType,ArrayType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pyspark.sql.utils
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer


creamos sesion para usar pyspark


In [2]:
SpSesion = SparkSession \
          .builder \
          .appName('etluser') \
          .getOrCreate() 

In [3]:
SpContext = SpSesion.sparkContext

vamos a cargar datos


In [4]:
path = 'australian_users_items.json'

In [5]:
dataset = SpSesion.read.json(path)

Vamos a pasar a explorar el archivo y ver de que se trata, tipos de columnas variables  etc


In [6]:
print('los tipos de datos son para cada columna \n',dataset.dtypes)
print()
print(f'tiene {len(dataset.columns)} columnas ')
print('las columnas son \n',dataset.columns)
print()
print(f'el dataset tiene {dataset.count()} filas')
print()



los tipos de datos son para cada columna 
 [('_corrupt_record', 'string'), ('items', 'array<struct<item_id:string,item_name:string,playtime_2weeks:bigint,playtime_forever:bigint>>'), ('items_count', 'bigint'), ('steam_id', 'string'), ('user_id', 'string'), ('user_url', 'string')]

tiene 6 columnas 
las columnas son 
 ['_corrupt_record', 'items', 'items_count', 'steam_id', 'user_id', 'user_url']

el dataset tiene 88310 filas



In [7]:
dataset.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- item_name: string (nullable = true)
 |    |    |-- playtime_2weeks: long (nullable = true)
 |    |    |-- playtime_forever: long (nullable = true)
 |-- items_count: long (nullable = true)
 |-- steam_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_url: string (nullable = true)



limpiamos la columna de  _corrupt records

In [8]:
dataset = dataset.drop("_corrupt_record") 

In [9]:
print('los tipos de datos son para cada columna \n',dataset.dtypes)
print()
print(f'tiene {len(dataset.columns)} columnas ')
print('las columnas son \n',dataset.columns)
print()
print(f'el dataset tiene {dataset.count()} filas')
print()


los tipos de datos son para cada columna 
 [('items', 'array<struct<item_id:string,item_name:string,playtime_2weeks:bigint,playtime_forever:bigint>>'), ('items_count', 'bigint'), ('steam_id', 'string'), ('user_id', 'string'), ('user_url', 'string')]

tiene 5 columnas 
las columnas son 
 ['items', 'items_count', 'steam_id', 'user_id', 'user_url']



el dataset tiene 88310 filas



a Pesar de existir corrupt records... no hemos perdido informacio!!!!!


In [10]:
dataset.printSchema()

root
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- item_name: string (nullable = true)
 |    |    |-- playtime_2weeks: long (nullable = true)
 |    |    |-- playtime_forever: long (nullable = true)
 |-- items_count: long (nullable = true)
 |-- steam_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_url: string (nullable = true)



In [11]:
print(dataset.show(10,truncate=False))

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

ahora convertimos el array struct de items  a un dataframe llamado df_items

esta seria una forma ...... peero podemos asignar una nueva columna a cada elemento     del array 

In [12]:

df_items = dataset.select(explode(col("items")).alias("item"))


df_items = df_items.select("item.*")


df_items.show(truncate=False)

+-------+---------------------------------+---------------+----------------+
|item_id|item_name                        |playtime_2weeks|playtime_forever|
+-------+---------------------------------+---------------+----------------+
|10     |Counter-Strike                   |0              |6               |
|20     |Team Fortress Classic            |0              |0               |
|30     |Day of Defeat                    |0              |7               |
|40     |Deathmatch Classic               |0              |0               |
|50     |Half-Life: Opposing Force        |0              |0               |
|60     |Ricochet                         |0              |0               |
|70     |Half-Life                        |0              |0               |
|130    |Half-Life: Blue Shift            |0              |0               |
|300    |Day of Defeat: Source            |0              |4733            |
|240    |Counter-Strike: Source           |0              |1853            |

al tener un array de structs  debo es una froma de ver el contenido

In [13]:
df = dataset.select("user_id", "user_url","steam_id", explode("items").alias ("items"))

In [14]:
df.show()

+-----------------+--------------------+-----------------+--------------------+
|          user_id|            user_url|         steam_id|               items|
+-----------------+--------------------+-----------------+--------------------+
|76561197970982479|http://steamcommu...|76561197970982479|{10, Counter-Stri...|
|76561197970982479|http://steamcommu...|76561197970982479|{20, Team Fortres...|
|76561197970982479|http://steamcommu...|76561197970982479|{30, Day of Defea...|
|76561197970982479|http://steamcommu...|76561197970982479|{40, Deathmatch C...|
|76561197970982479|http://steamcommu...|76561197970982479|{50, Half-Life: O...|
|76561197970982479|http://steamcommu...|76561197970982479|{60, Ricochet, 0, 0}|
|76561197970982479|http://steamcommu...|76561197970982479|{70, Half-Life, 0...|
|76561197970982479|http://steamcommu...|76561197970982479|{130, Half-Life: ...|
|76561197970982479|http://steamcommu...|76561197970982479|{300, Day of Defe...|
|76561197970982479|http://steamcommu...|

Desmenuzamos la columna Items y procedemos a borrarla

In [15]:
df = df.withColumn("item_id", col("items.item_id"))
df = df.withColumn("item_name", col("items.item_name"))
df = df.withColumn("playtime_2weeks", col("items.playtime_2weeks"))
df = df.withColumn("playtime_forever", col("items.playtime_forever"))

Ahora si lo tenemos listo para trabajar        

In [16]:
df = df.drop("items")

In [17]:
df.show(truncate=False)

+-----------------+----------------------------------------------------+-----------------+-------+---------------------------------+---------------+----------------+
|user_id          |user_url                                            |steam_id         |item_id|item_name                        |playtime_2weeks|playtime_forever|
+-----------------+----------------------------------------------------+-----------------+-------+---------------------------------+---------------+----------------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479|76561197970982479|10     |Counter-Strike                   |0              |6               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479|76561197970982479|20     |Team Fortress Classic            |0              |0               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479|76561197970982479|30     |Day of Defeat                    |0              |7               |
|765

Agregamos un espacio al final de cada fila

In [18]:
df = df.withColumn("user_url", concat(col("user_url"), lit(" ")))

In [19]:
df.show(truncate=False)

+-----------------+-----------------------------------------------------+-----------------+-------+---------------------------------+---------------+----------------+
|user_id          |user_url                                             |steam_id         |item_id|item_name                        |playtime_2weeks|playtime_forever|
+-----------------+-----------------------------------------------------+-----------------+-------+---------------------------------+---------------+----------------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|10     |Counter-Strike                   |0              |6               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|20     |Team Fortress Classic            |0              |0               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|30     |Day of Defeat                    |0              |7               

Creamos un nuevo dataset el cual no tendra los juegos que hayan sido descargados pero nunca jugados  ya que carece de sentido y nos permite achicar  la base de datos

In [22]:
df = df.filter(col("playtime_forever") != 0)

In [23]:
print('los tipos de datos son para cada columna \n',df.dtypes)
print()
print(f'tiene {len(df.columns)} columnas ')
print('las columnas son \n',df.columns)
print()
print(f'el dataset tiene {df.count()} filas')
print()

los tipos de datos son para cada columna 
 [('user_id', 'string'), ('user_url', 'string'), ('steam_id', 'string'), ('item_id', 'string'), ('item_name', 'string'), ('playtime_2weeks', 'bigint'), ('playtime_forever', 'bigint')]

tiene 7 columnas 
las columnas son 
 ['user_id', 'user_url', 'steam_id', 'item_id', 'item_name', 'playtime_2weeks', 'playtime_forever']



el dataset tiene 3281050 filas



In [25]:
df.show(truncate=False)

+-----------------+-----------------------------------------------------+-----------------+-------+----------------------------------+---------------+----------------+
|user_id          |user_url                                             |steam_id         |item_id|item_name                         |playtime_2weeks|playtime_forever|
+-----------------+-----------------------------------------------------+-----------------+-------+----------------------------------+---------------+----------------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|10     |Counter-Strike                    |0              |6               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|30     |Day of Defeat                     |0              |7               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|300    |Day of Defeat: Source             |0              |4733      

Eliminamos la columna de playtime_2weeks por que no vamos a utilizarla

In [26]:
df = df.drop("playtime_2weeks")

In [27]:
df.show(truncate=False)

+-----------------+-----------------------------------------------------+-----------------+-------+----------------------------------+----------------+
|user_id          |user_url                                             |steam_id         |item_id|item_name                         |playtime_forever|
+-----------------+-----------------------------------------------------+-----------------+-------+----------------------------------+----------------+
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|10     |Counter-Strike                    |6               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|30     |Day of Defeat                     |7               |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |76561197970982479|300    |Day of Defeat: Source             |4733            |
|76561197970982479|http://steamcommunity.com/profiles/76561197970982479 |765611979709824

Eliminamos 2 columnas que no vamos a utilizar, ya que no vamos a utilizar merge para la FastApi   y trabajaremos independientemente cada Base de datos

In [28]:
df = df.drop('user_url')
df = df.drop('steam_id')

In [29]:
df.show(truncate=False)

+-----------------+-------+----------------------------------+----------------+
|user_id          |item_id|item_name                         |playtime_forever|
+-----------------+-------+----------------------------------+----------------+
|76561197970982479|10     |Counter-Strike                    |6               |
|76561197970982479|30     |Day of Defeat                     |7               |
|76561197970982479|300    |Day of Defeat: Source             |4733            |
|76561197970982479|240    |Counter-Strike: Source            |1853            |
|76561197970982479|3830   |Psychonauts                       |333             |
|76561197970982479|2630   |Call of Duty 2                    |75              |
|76561197970982479|3900   |Sid Meier's Civilization IV       |338             |
|76561197970982479|3920   |Sid Meier's Pirates!              |2               |
|76561197970982479|6400   |Joint Task Force                  |286             |
|76561197970982479|6910   |Deus Ex: Game

In [53]:
# Contar los 'item_name'
conteo_item_name_spark = df.groupBy('item_name').count()

# Contar los duplicados
duplicados_spark = df.count() - df.distinct().count()

# Contar los nulos
nulos_spark = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

print(conteo_item_name_spark)

print(duplicados_spark)

print(nulos_spark)

DataFrame[item_name: string, count: bigint]
38884
DataFrame[user_id: bigint, item_id: bigint, item_name: bigint, playtime_forever: bigint]


Pasamos el DF a Pandas

In [30]:
df_pandas = df.toPandas()

In [38]:
df_pandas

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,30,Day of Defeat,7
2,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,3830,Psychonauts,333
...,...,...,...,...
3281045,76561198329548331,304930,Unturned,677
3281046,76561198329548331,227940,Heroes & Generals,43
3281047,76561198329548331,388490,One Way To Die: Steam Edition,3
3281048,76561198329548331,521570,You Have 10 Seconds 2,4


Creamos un CSV final con el ETL Realizado

In [56]:
#df_pandas.to_csv('user_items_final.csv', index=False)

In [52]:

conteo_item_name = df_pandas['item_name'].count()

# Contar los duplicados
duplicados = df_pandas.duplicated().sum()

# Contar los nulos
nulos = df_pandas.isnull().sum()
print (duplicados,nulos)

38884 user_id             0
item_id             0
item_name           0
playtime_forever    0
dtype: int64


In [40]:
tipo_datos = df_pandas.dtypes
print(tipo_datos)

user_id             object
item_id             object
item_name           object
playtime_forever     int64
dtype: object


In [46]:

df_pandas['item_id'] = df_pandas['item_id'].astype('int64')
df_pandas['item_name'] = df_pandas['item_name'].astype('string')

In [45]:
tipo_datos = df_pandas.dtypes
print(tipo_datos)

user_id                     object
item_id                      int32
item_name           string[python]
playtime_forever             int64
dtype: object
